# Point Visibility Maps: Experiments on Sample Images

##  Information Visualization on Mobile Devices: Efficient Occlusion Detection for Geospatial Applications

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
from pathlib import Path
from typing import List
import time
from collections import defaultdict

import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm

sys.path.insert(0, '..')

from outdoorar.constants import RESOURCES_DIR, CAMERAS_DIR, OUTPUT_DIR, FIGURES_DIR, PROJECT_DIR, get_visibility_dir
from outdoorar.rendering import get_image_coordinates, is_inside_image
from outdoorar.visibility import Visibility, Vertex, from_json, calculate_visibility, NearestNeighborSelector
from outdoorar.sphere_sampling import SamplingScheme

## Input data

In [ ]:
captured_images = RESOURCES_DIR.joinpath('capturedImages')
cameras_sfm = CAMERAS_DIR.joinpath('cameras.sfm')

In [ ]:
n_range = [2,4,8,16,32]

## Functions

In [ ]:
def read_polylines(n, sampling_scheme, parent_visibility_folder=None):
    
    if parent_visibility_folder is None:
        parent_visibility_folder = get_visibility_dir(sampling_scheme)
        
    visibility_folder = parent_visibility_folder.joinpath(f'n_{n}')
    blue_visibility = from_json(visibility_folder.joinpath('BluePolyline.json'))
    red_visibility = from_json(visibility_folder.joinpath('RedPolyline.json'))
    yellow_visibility = from_json(visibility_folder.joinpath('YellowPolyline.json'))
    green_visibility = from_json(visibility_folder.joinpath('GreenPolyline.json'))

    polylines = [
        (green_visibility, (0, 1, 0.5)), 
        (yellow_visibility, (1, 1, 0)),
        (red_visibility, (1, 0, 0)),
        (blue_visibility, (0, 170/255, 1)),
    ]
    return polylines

In [ ]:
def get_or_create_output_directory(n, sampling_scheme, algorithm, parent_output_directory=OUTPUT_DIR):
    output_directory = parent_output_directory.joinpath(
        f'{sampling_scheme.name.lower()}_{algorithm.name.lower()}', 
        f'n_{n}',
    )
    output_directory.mkdir(exist_ok=True, parents=True)
    return output_directory

In [ ]:
def polyline_to_matrix(visibility: Visibility) -> np.ndarray:
    return np.array([[v.x, v.y, v.z] for v in visibility.vertices])

In [ ]:
def clip(coords: np.ndarray, image_width: int, image_height: int) -> np.ndarray:
    return np.maximum(np.minimum(coords, np.array([[image_width], [image_height], [1]])), 0)

In [ ]:
def plot_polyline(image_coords, node_visibility, edges, color, image_width, image_height):
    if not np.any(node_visibility):
        return 
    plt.scatter(
        x=image_coords[:, node_visibility][0], 
        y=image_coords[:, node_visibility][1], 
        facecolors=color,
        edgecolors=color
    )
    for edge in edges:
        v1_visible = node_visibility[edge.vertex1]
        v2_visible = node_visibility[edge.vertex2]
        coords = clip(image_coords[:, [edge.vertex1, edge.vertex2]], image_width, image_height)
        if v1_visible and v2_visible:
            plt.plot(coords[0], coords[1], '-', c=color)
        elif v1_visible or v2_visible:
            plt.plot(coords[0], coords[1], '--', c=color)


## Camera information

In [ ]:
cameras = json.load(cameras_sfm.open('r'))

In [ ]:
cameras.keys()

### Camera intrinsic matrix

In [ ]:
intrinsic = cameras['intrinsics'][0]

In [ ]:
K = np.array([
    [float(intrinsic["pxFocalLength"]), 0, float(intrinsic["principalPoint"][0]), 0],
    [0, float(intrinsic["pxFocalLength"]), float(intrinsic["principalPoint"][1]), 0],
    [0, 0, 1, 0]
])

In [ ]:
K

### Sample images

In [ ]:
views = {view['poseId'] : {
    'imgName': view['path'][view['path'].rfind('/')+1:].upper(),
    'width': int(view['width']),
    'height': int(view['height'])
} for view in cameras['views']}    

### Prepare output dataframe with visibility information

In [ ]:
def get_output_dataframe(polylines, views):
    annotations_info: list[tuple[str, str]] = []

    for curr_polyline, curr_color in polylines:
        poly_name = curr_polyline.name
        num_vertices = len(curr_polyline.vertices)
        annotations_info.extend(list(zip([poly_name] * num_vertices, [str(x) for x in range(num_vertices)])))
        
    images_index = [view['imgName'] for view in views.values()]
    
    return pd.DataFrame(
        data=0, 
        columns=pd.MultiIndex.from_tuples(annotations_info), 
        index=images_index, 
        dtype=int,
    )

### Draw visible annotations in the image

In [ ]:
def get_extrinsic_matrix(rotation, center):
    R = np.array([float(x) for x in rotation]).reshape((3,3), order='F')
    C = np.array([[float(x)] for x in center])
    T = - np.matmul(R, C)
    return np.vstack((np.hstack((R, T)), np.array([0, 0, 0, 1])))

In [ ]:
def draw_visible_annotations(n, sampling_scheme, algorithm):
    polylines = read_polylines(n, sampling_scheme)
    output_directory = get_or_create_output_directory(n, sampling_scheme, algorithm)
    results_df = get_output_dataframe(polylines, views)
    times = []

    for pose_obj in tqdm(cameras['poses'], desc=f'n={n}'):

        # get camera pose and image details
        pose = pose_obj['pose']['transform']
        view = views[pose_obj['poseId']]
        img_name = view['imgName']
        image_width, image_height = view['width'], view['height']

        # plot image
        im = plt.imread(captured_images.joinpath(img_name))
        fig, ax = plt.subplots(figsize=(16,12))
        implot = ax.imshow(im)
        plt.axis('off')

        # calculate extrinsic matrix
        M = get_extrinsic_matrix(pose['rotation'], pose['center'])
        eye = [float(x) for x in pose["center"]]  

        image_time = 0
        # for each polyline calculate visibility
        for curr_polyline, curr_color in polylines:
            start = time.time()
            polyline_matrix = polyline_to_matrix(curr_polyline)
            curr_image_coords = get_image_coordinates(polyline_matrix, K, M)
            curr_is_visible = np.logical_and(
                calculate_visibility(curr_polyline.vertices, eye, sampling_scheme, algorithm),
                is_inside_image(curr_image_coords, image_width, image_height)
            )
            image_time += (time.time() - start)
            results_df.loc[img_name, results_df.columns.get_level_values(0)==curr_polyline.name] = curr_is_visible.astype(int)
            # plot visible points of the polyline to the image
            plot_polyline(curr_image_coords, curr_is_visible, curr_polyline.edges, curr_color, image_width, image_height)

        plt.savefig(output_directory.joinpath(img_name), bbox_inches='tight')
        plt.close()
        times.append(image_time)

    results_df.to_csv(output_directory.joinpath('visibility.csv'))
    return times

In [ ]:
algorithms = {
    SamplingScheme.EQUAL_ANGLE: [
        NearestNeighborSelector.EQUAL_SPACING, 
        NearestNeighborSelector.COSINE_DISTANCE,
    ],
    SamplingScheme.GOLDEN_SPIRAL: [
        NearestNeighborSelector.COSINE_DISTANCE,
    ],
}

In [ ]:
times = defaultdict(list)

for sampling_scheme in SamplingScheme:
    for algorithm in algorithms[sampling_scheme]:
        for n in n_range:
            times[(sampling_scheme.name, algorithm.name, n)].extend(
                draw_visible_annotations(n, sampling_scheme, algorithm)
            )


In [ ]:
for key, values in times.items():
    print(key, np.mean(values))

### Compare to ground truth

In [ ]:
ground_truth_file_path = RESOURCES_DIR.joinpath('ground_truth.csv')
gt_df = pd.read_csv(ground_truth_file_path, header=[0,1], index_col=0)

In [ ]:
def get_scores(gt_df, results_df, cols=None):
    if cols is None:
        cols = gt_df.columns
    tp = np.logical_and(gt_df == 1, results_df[cols] == 1).sum().sum()
    tn = np.logical_and(gt_df == 0, results_df[cols] == 0).sum().sum()
    fp = np.logical_and(gt_df == 0, results_df[cols] == 1).sum().sum()
    fn = np.logical_and(gt_df == 1, results_df[cols] == 0).sum().sum()
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = (2*tp) / (2*tp + fp + fn)
    return accuracy, precision, recall, f1    

In [ ]:
headers = [
    f"{sampling.name.replace('_', ' ').title()} - {algorithm.name.replace('_', ' ').title()}"
    for sampling in SamplingScheme for algorithm in algorithms[sampling] 
]


In [ ]:
scores = pd.DataFrame(
    data=np.nan, 
    columns=pd.MultiIndex.from_product(
        [
            headers, 
            ['Accuracy', 'Precision', 'Recall', 'F1-Score'],
        ]
    ), 
    index=pd.Index(n_range, name='Number of viewpoints')
)


for n in n_range:
    row = []
    for sampling_scheme in SamplingScheme:
        for algorithm in algorithms[sampling_scheme]:
            results_df = pd.read_csv(
                get_or_create_output_directory(n, sampling_scheme, algorithm).joinpath('visibility.csv'),
                index_col=0,
                header=[0,1]
            )
            row.extend(get_scores(gt_df, results_df))
    scores.loc[n] = row

In [ ]:
scores

## Smaller mesh experiment

In [ ]:
smaller_mesh_1024 = pd.read_csv(
    RESOURCES_DIR.joinpath('decimatedMesh_closedHoles_1024.csv'), 
    header=[0,1], 
    index_col=0,
)

In [ ]:
scores_1024 = get_scores(gt_df, smaller_mesh_1024)
print(f'{scores_1024[0]:.4f} & {scores_1024[3]:.4f}')

In [ ]:
smaller_mesh_2048 = pd.read_csv(
    RESOURCES_DIR.joinpath('decimatedMesh_closedHoles_2048.csv'), 
    header=[0,1], 
    index_col=0,
)

In [ ]:
scores_2048 = get_scores(gt_df, smaller_mesh_2048)
print(f'{scores_2048[0]:.4f} & {scores_2048[3]:.4f}')

In [ ]:
smaller_mesh_4096 = pd.read_csv(
    RESOURCES_DIR.joinpath('decimatedMesh_closedHoles_4096.csv'), 
    header=[0,1], 
    index_col=0,
)

In [ ]:
scores_4096 = get_scores(gt_df, smaller_mesh_4096)
print(f'{scores_4096[0]:.4f} & {scores_4096[3]:.4f}')